In [ ]:
import numpy as np
import pandas as pd
import math
import scipy
import matplotlib.pyplot as plt
from scipy.stats import norm,binom
from scipy.io import loadmat
from scipy.special import digamma, gammaln, gamma
from numpy.linalg import inv
#import cv2
from scipy import ndimage
import tensorflow as tf
import os, sys

In [ ]:
from utils.data_load import Image_data

In [ ]:
image_data = Image_data(npyfilename1 = 'PASCAL_data.npy',npyfilename2 = 'gray_scale.npy',
                        kernel_file_name = 'kernel1.mat')

In [ ]:
image_data.original_images.shape

In [ ]:
image_data.normalise()

In [ ]:
plt.imshow(image_data.conv_images[48542],cmap = 'gray')
plt.show()


In [ ]:
plt.imshow(image_data.original_images[48542],cmap = 'gray')
plt.show()